## Load in R Packages

In [ ]:
library("Eunomia")
library("DatabaseConnector")
library("CohortMethod")

## Connect to Eunomia GiBleed dataset (default) and Test a Query

In [ ]:
# Got an error once and had to do this to resolve...
#options(databaseConnectorInteger64AsNumeric = FALSE)


connectionDetails <- getEunomiaConnectionDetails()


createCohorts(connectionDetails)
connection <- DatabaseConnector::connect(connectionDetails)

# This is how you would connect to a different dataset
#conn <- DatabaseConnector::connect(dbms = "sqlite", server = getDatabaseFile("MIMIC"))

# Get top ppl (use limit in sqlite, not top)
DatabaseConnector::querySql(connection, sql="SELECT * FROM person;")

# Another example query...
#DatabaseConnector::getTableNames(connection,databaseSchema = 'main')



## Do Cohort Method Exercise from Book of Ohdsi
[Exercise 12.1](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple1)

In [ ]:
nsaids <- c(1118084, 1124300) # celecoxib, diclofenac
covSettings <- createDefaultCovariateSettings(
  excludedCovariateConceptIds = nsaids,
  addDescendantsToExclude = TRUE)

# Load data:
cmData <- getDbCohortMethodData(
  connectionDetails = connectionDetails,
  cdmDatabaseSchema = "main",
  targetId = 1,
  comparatorId = 2,
  outcomeIds = 3,
  exposureDatabaseSchema = "main",
  exposureTable = "cohort",
  outcomeDatabaseSchema = "main",
  outcomeTable = "cohort",
  covariateSettings = covSettings)
summary(cmData)

## Create Study Population 
Book of OHDSI [Exercise 12.2](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple2)

In [ ]:
studyPop <- createStudyPopulation(
  cohortMethodData = cmData,
  outcomeId = 3,
  washoutPeriod = 180,
  removeDuplicateSubjects = "remove all",
  removeSubjectsWithPriorOutcome = TRUE,
  riskWindowStart = 0,
  startAnchor = "cohort start",
  riskWindowEnd = 99999)
drawAttritionDiagram(studyPop)

## Cox Regression
Book of OHDSI [Exercise 12.3](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple3)

In [ ]:
model <- fitOutcomeModel(population = studyPop,
                         modelType = "cox")
model

## Propensity Model
Book of OHDSI [Exercise 12.4](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple4)

In [ ]:
ps <- createPs(cohortMethodData = cmData,
               population = studyPop)
plotPs(ps, showCountsLabel = TRUE, showAucLabel = TRUE)

## Disconnect

In [ ]:

DatabaseConnector::disconnect(connection)